In [1]:
import itertools
def num_overlap(a, b):
    return sum([x==y for x, y in zip(a,b)])

def filter_valid(perms, guesses, counts):
    retval = []
    for perm in perms:
        still_valid = True
        for count, guess in zip(counts, guesses):
            n_count = num_overlap(perm, guess)
            if n_count != count:
                still_valid = False
        if still_valid:
            retval.append(perm)
    return retval

memoize = {}
def optimal_strategy(perms):
    """
    Use mini-max set exclusion like knuth
    """
    if len(perms) == 1:
        return 1, perms[0]
    perms = sorted(perms)
    if tuple(perms) in memoize:
        return memoize[tuple(perms)]
    guesses = []
    for guess in perms:
        lengths = []    
        for code in perms:
            count = num_overlap(guess, code)
            l = filter_valid(perms, [guess], [count])
            if len(l) == len(perms):
                continue
            score = 1 + optimal_strategy(l)[0]
            lengths.append((score, guess))
        guesses.append(max(lengths, key=lambda x: x[0]))
    retval = min(guesses, key=lambda x: x[0])
    memoize[tuple(perms)] = retval
    return retval
                    
        

In [2]:
def play_game(secret_code, perms, guesses, counts):
    if len(guesses ) > 0 and guesses[-1] == secret_code:
        return 0, list(guesses)
    guess = optimal_strategy(perms)[1]
    guesses.append(guess)
    count = num_overlap(guesses[-1], secret_code)
    counts.append(count)
    l = filter_valid(perms, guesses, counts)
    retval, ret_list = play_game(secret_code, l, guesses, counts)
    return retval + 1, ret_list

In [3]:
perms = [x for x in itertools.permutations(range(4))]
games = []
for perm in perms:
    game = play_game(perm, perms, [], []), perm
    games.append(game)

In [4]:
print(max(games, key=lambda x: x[0]))

((6, [(0, 1, 2, 3), (0, 2, 3, 1), (0, 3, 1, 2), (1, 2, 0, 3), (2, 1, 3, 0), (3, 0, 2, 1)]), (3, 0, 2, 1))


In [5]:
max(memoize.values(), key=lambda x: x[0])
    

(6, (0, 1, 2, 3))